# 460. LFU Cache
#### https://leetcode.com/problems/lfu-cache/

# Approach 1: Hash Table + Double Linked List

In [7]:
class DListNode(object):
    def __init__(self, key, val, freq):
        self.next = None
        self.prev = None
        self.key = key
        self.val = val
        self.freq = freq

class LFUCache(object):

    def __init__(self, capacity):
        """
        :type capacity: int
        """
        self.capacity = capacity
        self.head = DListNode(-1, -1, 0)
        self.tail = DListNode(-1, -1, 0)
        self.head.next = self.tail
        self.tail.prev = self.head
        self.key_to_node = {}
        self.freq_to_node = {0: self.tail}
        
    def remove(self, node):
        if node.freq != node.prev.freq and node.freq != node.next.freq:
            del self.freq_to_node[node.freq]
        elif self.freq_to_node[node.freq] == node:
            self.freq_to_node[node.freq] = node.prev
        node.prev.next = node.next
        node.next.prev = node.prev
        
    def insert(self, node_ref, node):
        node.prev = node_ref
        node.next = node_ref.next
        node_ref.next = node
        node.next.prev = node
        
    def update(self, node):
        p = node.prev
        self.remove(node)
        node.freq += 1
        if node.freq in self.freq_to_node:
            self.insert(self.freq_to_node[node.freq], node)
        elif node.freq - 1 in self.freq_to_node:
            self.insert(self.freq_to_node[node.freq - 1], node)
        else:
            self.insert(p, node)    
        self.freq_to_node[node.freq] = node 
        
    def get(self, key):
        """
        :type key: int
        :rtype: int
        """
        if self.capacity <= 0: return -1
        if key not in self.key_to_node:
            return -1
        
        node = self.key_to_node[key]
        self.update(node)

        return node.val
        

    def put(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: None
        """
        if self.capacity <= 0: return
        if key not in self.key_to_node:
            if len(self.key_to_node) == self.capacity:
                node = self.head
                del self.key_to_node[self.head.next.key]
                self.remove(self.head.next)
                
            node = DListNode(key, value, 1)
            node_ref = self.freq_to_node[1] if 1 in self.freq_to_node else self.head
            self.insert(node_ref, node)
            self.key_to_node[key] = node
            self.freq_to_node[1] = node
            return

        node = self.key_to_node[key]
        node.val = value
        self.update(node)

# Your LFUCache object will be instantiated and called as such:
# obj = LFUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

# Verification

In [8]:
testcases = [(["LFUCache","put","get","put","get","get"],[[1],[2,1],[2],[3,2],[2],[3]])]
expected = [[None,None,1,None,-1,2]]

for case in zip(testcases, expected):
    outputs = [None]
    sol = LFUCache(case[0][1][0][0])
    print("Input:    " + str(case[0]))
    funcs = case[0][0]
    params = case[0][1]
    for i, func in enumerate(funcs):
        if i: param = params[i]
        if func == "put":
            outputs.append(sol.put(param[0], param[1]))
        if func == "get":
            outputs.append(sol.get(param[0]))
    print("Output:   " + str(outputs))
    print("Expected: " + str(case[1]) + "\n")

Input:    (['LFUCache', 'put', 'get', 'put', 'get', 'get'], [[1], [2, 1], [2], [3, 2], [2], [3]])
Output:   [None, None, 1, None, -1, 2]
Expected: [None, None, 1, None, -1, 2]

